# Preparations

## Import modules

In [124]:
from IPython.display import display, Markdown
import time
from datetime import timedelta
from datetime import date
import os
import pathlib

from sqlalchemy.engine import *
from sqlalchemy import text

import ebookmeta
import zipfile
from lxml import etree
from PIL import Image

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import seaborn as sns

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.tree import export_graphviz
# from sklearn import tree

## Define functions

In [128]:
def get_credentials(filename):
    credential = open(filename,'r')
    
    DIALECT = 'mysql'
    SQL_DRIVER = 'pymysql'
    USERNAME = credential.readline().rstrip()
    PASSWORD = credential.readline().rstrip()
    HOST = credential.readline().rstrip()
    PORT = credential.readline().rstrip()
    SERVICENAME = 'sdfs'
    ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD + '@' + HOST + ':' + str(PORT) #+ '/?service_name=' + SERVICENAME
    credential.close()
    return USERNAME,PASSWORD,HOST,PORT,SERVICENAME,ENGINE_PATH_WIN_AUTH

def connect_oracle(credentials):
    connection = cx_Oracle.connect(credentials[0], credentials[1], '{}:{}/{}'.format(credentials[2],credentials[3],credentials[4]))
    return connection

def connect_sqlalchemy(credentials):
    ENGINE = create_engine(credentials[-1])
    return ENGINE

def query_to_excel(query,fname):
    start_time = time.time()
    if os.path.isfile(fname) == True:
        print('File already exists.')
    else:
        # Set credential
        my_credentials = get_credentials('my_credentials.txt')
        
        # Open
        # my_connection = connect_oracle(my_credentials)
        engine = connect_sqlalchemy(my_credentials)
        df = pd.read_sql_query(query,engine)
        df.columns = df.columns.str.upper()
        
        # Convert dataframe to excel and save
        with pd.ExcelWriter(fname) as writer:
            df.to_excel(writer, index=False)
            
        # Close
        # my_connection.close()
        engine.dispose()
        
    end_time = time.time()
    print('Execution time: {:2f}'.format(end_time - start_time),'s')

def excel_to_sql(fname,schema,table_name):
    start_time = time.time()
    
    # Set credential
    my_credentials = get_credentials('my_credentials.txt')
    
    # Open
    engine = connect_sqlalchemy(my_credentials)
    
    dataset_dir = pathlib.Path(r'D:/Dataset')
    dataset_dir_iter = dataset_dir.rglob("*")
    dataset_dir_list = [item for item in dataset_dir_iter if item.is_dir()]
    dataset_dir_list
    fullpath = os.path.join(str(dataset_dir_list[1]),fname)
    is_fullpath = os.path.exists(os.path.join(str(dataset_dir_list[1]),fname))
    if is_fullpath == True:
        df = pd.read_csv(fullpath)
    
    # Convert dataframe to sql
    sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False)
        
    # Close
    engine.dispose()
        
    print('Fullpath exists?',is_fullpath)
    print('Fullpath:',fullpath)
    end_time = time.time()
    print('Execution time: {:2f}'.format(end_time - start_time),'s')

def query_check(schema,table_name):
    query = f'''
                select count(*) num_cases
                from information_schema.tables
                where 1=1
                      and upper(table_schema) = "{schema.upper()}"
                      and upper(table_name) = "{table_name.upper()}"
              '''
    return query

def item_check(schema,table_name,metadata):
    query = f'''
                select
                       1
                      ,count(*) num_cases
                from {schema}.{table_name}
                where 1=1
                      and 1=1
                group by
                       1
              '''
    return query

def merge_insert(schema,table_name,metadata):
    query = f'''
                merge into {schema}.{table_name} dm1
                using
                (
                ) on (dm1.primary_key = dm2.primary_key)
                when matched then
                update
                   set gyjgjkbkjnkjnmjbkjhjk
                when not matched then
                insert(
                      )
                values(
                      )
                commit;
              '''
    return query

def create_table_sql(query,fname,schema,table_name):
    start_time = time.time()
    # Set credential
    my_credentials = get_credentials('my_credentials.txt')

    special_characters = "!@#$%^&*()[]{};:,./<>?\|`~-=_+"

    filename = fname.split('.')
    fname_clean = filename[0].translate ({ord(c): "_" for c in special_characters})
    for i in range(1,len(filename)):
        fname_clean += '.' + filename[i]
    
    # Replace special characters in table name with "_"
    if any(item in special_characters for item in table_name):
        print('Replacing special characters in table name with "_".')
        table_name = table_name.translate ({ord(c): "_" for c in special_characters})
    
    dataset_dir = pathlib.Path(r'D:/Dataset')
    dataset_dir_list = [item for item in dataset_dir.rglob("*") if item.is_dir()]
    dataset_file_list = [item for item in dataset_dir.rglob("*") if item.is_file()]
    dataset_dir_array_shape = np.array(dataset_dir_list).shape
    dataset_file_array_shape = np.array(dataset_file_list).shape
    fullpath = None
    is_fullpath = None
    for i in range(dataset_dir_array_shape[0]):
        for j in range(dataset_file_array_shape[0]):
            if os.path.join(str(dataset_dir_list[i]),fname) == str(dataset_file_list[j]):
                folder_path = str(dataset_dir_list[i])
                fullpath = os.path.join(str(dataset_dir_list[i]),fname)
                is_fullpath = os.path.exists(fullpath)
                if table_name == '' or table_name == None:
                    name = fname.split('.')
                    filename = name[0].split('/')
                    table_name = filename[-1]
                if query == '' or query == None:
                    query = f'''
                                select * from {schema}.{table_name};
                             '''
                fname = folder_path.replace("\\","/")+'/'+fname
            elif os.path.join(str(dataset_dir_list[i]),fname_clean) == str(dataset_file_list[j]):
                folder_path = str(dataset_dir_list[i])
                fullpath = os.path.join(str(dataset_dir_list[i]),fname_clean)
                is_fullpath = os.path.exists(fullpath)
                if table_name == '' or table_name == None:
                    name = fname_clean.split('.')
                    filename = name[0].split('/')
                    table_name = filename[-1]
                if query == '' or query == None:
                    query = f'''
                                select * from {schema}.{table_name};
                             '''
                fname_clean_path = folder_path.replace("\\","/")+'/'+fname_clean
                
    # Replace special characters in file name with "_"
    if any(item in special_characters for item in fname) and os.path.isfile(fname_clean_path) == False:
        print('File name contains special characters. Renaming file.')
        old_file = pathlib.Path(os.path.join(folder_path,fname))
        new_file = pathlib.Path(os.path.join(folder_path,fname_clean))
        fname = str(old_file.rename(new_file))
        fullpath = fname
    else:
        fname = fname_clean_path
        
    if os.path.isfile(fname) == True:
        try:
            print('File already exists.')
            print('Fullpath:',fullpath)
        except Error as Err:
            print(Err)
        else:            
            # Open
            engine = connect_sqlalchemy(my_credentials)
    
            conn = engine.raw_connection()
            curr = conn.cursor()
            
            # Check if table exists in database
            print('Checking in the database.')
            try:
                curr.execute(query_check(schema,table_name))
            except Error as Err:
                print(Err)
            else:
                if curr.fetchone()[0] >= 1:
                    df = pd.read_sql_query(query,engine)
                    df.columns = df.columns.str.upper()
                    is_new_table = input('Table already exists in the database. Do you want to create a new table? [YES/NO]')
                    if is_new_table.upper() == 'YES' or is_new_table.upper() == 'Y':
                        new_table_name = input('Insert new table name.')
                        # Replace special characters in table name with "_"
                        if any(item in special_characters for item in new_table_name):
                            print('Replacing special characters in new table name with "_".')
                            new_table_name = new_table_name.translate ({ord(c): "_" for c in special_characters})
                        new_fname = folder_path.replace("\\","/")+'/'+new_table_name+''.join(pathlib.Path(fname).suffixes)
                        try:
                            curr.execute(query_check(schema,new_table_name))
                        except Error as Err:
                            print(Err)
                        else:
                            if curr.fetchone()[0] >= 1:
                                print('New table name already exists in the database.')
                            else:
                                print('New fullpath:',new_fname)
                                # Convert dataframe to excel and save
                                if fname.endswith('.xlsx') or fname.endswith('.xlsm') or fname.endswith('.xls'):
                                    with pd.ExcelWriter(new_fname) as writer:
                                        df.to_excel(writer, index=False)
                                elif fname.endswith('.csv'):
                                    df.to_csv(new_fname, index=False, encoding='utf-8')
                                curr.close()
                                # Convert dataframe to sql
                                print('Importing to database.')
                                sql = df.to_sql(name=new_table_name.upper(), con=engine, schema=schema, index=False)
                    else:
                        if_exists_proceed = input('Do you want to append to or replace the existing table? [YES/NO]')
                        if if_exists_proceed.upper() == 'YES' or if_exists_proceed.upper() == 'Y':
                            method = input('Choose on how to proceed: [APPEND/REPLACE]')
                            method.lower()
                            # Convert dataframe to sql
                            print('Importing to database.')
                            sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False, if_exists=method)
                else:
                    if is_fullpath == True:
                        print('No existing table. Importing to database.')
                        if fname.endswith('.xlsx') or fname.endswith('.xlsm') or fname.endswith('.xls'):
                            # Read file and open as dataframe
                            df = pd.read_excel(fullpath, sheet_name=None, index_col=False)
                            sheet_names = df.keys()
                            for i in range(len(sheet_names)):
                                table_name = table_name + '_' + sheet_names[i]
                                # Convert dataframe to sql
                                sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False)
                        elif fname.endswith('.csv'):
                            # Read file and open as dataframe
                            df = pd.read_csv(fullpath, index_col=False)
                            # Convert dataframe to sql
                            sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False)
        
            # Close
            engine.dispose()
            print('Finished.')
    else:
        # Open
        engine = connect_sqlalchemy(my_credentials)
        conn = engine.raw_connection()
        curr = conn.cursor()
        
        df = pd.read_sql_query(query,engine)
        df.columns = df.columns.str.upper()
        
        # Convert dataframe to excel and save
        print('Creating file.')
        if fname.endswith('.xlsx') or fname.endswith('.xlsm') or fname.endswith('.xls'):
            with pd.ExcelWriter(fname) as writer:
                df.to_excel(writer, index=False)
        elif fname.endswith('.csv'):
            df.to_csv(fname, index=False, encoding='utf-8')
            
        if is_fullpath == True:
            # Read file and open as dataframe
            if fname.endswith('.xlsx') or fname.endswith('.xlsm') or fname.endswith('.xls'):
                df = pd.read_excel(fullpath, sheet_name=None, index_col=False)
                sheet_names = df.keys()
            elif fname.endswith('.csv'):
                df = pd.read_csv(fullpath, index_col=False)

        # Check if table exists in database
        print('Checking in the database.')
        try:
            curr.execute(query_check(schema,table_name))
        except Error as Err:
            print(Err)
        else:
            if curr.fetchone()[0] >= 1:
                is_new_table = input('Table already exists in the database. Do you want to create a new table? [YES/NO]')
                if is_new_table.upper() == 'YES' or is_new_table.upper() == 'Y':
                    df = pd.read_sql_query(query,engine)
                    df.columns = df.columns.str.upper()
                    new_table_name = input('Insert new table name.')
                    # Replace special characters in table name with "_"
                    if any(item in special_characters for item in new_table_name):
                        print('Replacing special characters in new table name with "_".')
                        new_table_name = new_table_name.translate ({ord(c): "_" for c in special_characters})
                    new_fname = folder_path.replace("\\","/")+'/'+new_table_name+''.join(pathlib.Path(fname).suffixes)
                    try:
                        curr.execute(query_check(schema,new_table_name))
                    except Error as Err:
                        print(Err)
                    else:
                        if curr.fetchone()[0] >= 1:
                            print('New table name already exists in the database.')
                        else:
                            print('New fullpath:',new_fname)
                            # Convert dataframe to excel and save
                            if new_table_name.endswith('.xlsx') or fname.endswith('.xlsm') or fname.endswith('.xls'):
                                with pd.ExcelWriter(new_fname) as writer:
                                    df.to_excel(writer, index=False)
                            elif fname.endswith('.csv'):
                                df.to_csv(new_fname, index=False, encoding='utf-8')
                            curr.close()
                            # Convert dataframe to sql
                            print('Importing to database.')
                            sql = df.to_sql(name=new_table_name.upper(), con=engine, schema=schema, index=False)
                else:
                    if_exists_proceed = input('Do you want to append to or replace the existing table? [YES/NO]')
                    if if_exists_proceed.upper() == 'YES' or if_exists_proceed.upper() == 'Y':
                        method = input('Choose on how to proceed: [APPEND/REPLACE]')
                        method.lower()
                        # Convert dataframe to sql
                        print('Importing to database.')
                        sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False, if_exists=method)
            else:
                # Convert dataframe to sql
                print('No existing table. Importing to database.')
                if fname.endswith('.xlsx') or fname.endswith('.xlsm') or fname.endswith('.xls'):
                    for i in range(len(sheet_names)):
                        table_name = table_name + '_' + sheet_names[i]
                        sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False)
                elif fname.endswith('.csv'):
                    sql = df.to_sql(name=table_name, con=engine, schema=schema, index=False)
    
        # Close
        engine.dispose()
        print('Finished.')
        
    end_time = time.time()
    print('Execution time: {:2f}'.format(end_time - start_time),'s')




def epub_metadata_to_df(fname):
    epub_meta = ebookmeta.get_metadata(fname)
    meta_keys = ['identifier','title','author_list','series','series_index','tag_list','lang',
                 'description','translators','format','format_version','epub_cover_image_data',
                 'cover_media_type','cover_file_name','file','file_created','file_modified']
    
    epub_identifier = epub_meta.identifier
    epub_title = epub_meta.title
    epub_author_list = epub_meta.author_list
    epub_series = epub_meta.series
    epub_series_index = epub_meta.series_index
    epub_tag_list = epub_meta.tag_list
    epub_lang = epub_meta.lang
    epub_description = epub_meta.description
    try:
        epub_translators = epub_meta.translators
    except:
        epub_translators = []
    epub_format = epub_meta.format
    epub_format_version = epub_meta.format_version
    epub_cover_image_data = epub_meta.cover_image_data
    epub_cover_media_type = epub_meta.cover_media_type
    epub_cover_file_name = epub_meta.cover_file_name
    epub_file = epub_meta.file
    epub_file_created = epub_meta.file_created
    epub_file_modified = epub_meta.file_modified
    
    meta_values = [epub_identifier,epub_title,epub_author_list,epub_series,epub_series_index,
                   epub_tag_list,epub_lang,epub_description,epub_translators,epub_format,epub_format_version,
                   epub_cover_image_data,epub_cover_media_type,epub_cover_file_name,epub_file,epub_file_created,epub_file_modified]
    
    dict = {}
    for i in range(len(meta_keys)):
        if meta_keys[i] in dict:
            dict[i][0].append(meta_values[i])
        else:
            dict[meta_keys[i]] = meta_values[i]
    df_epub = pd.DataFrame.from_dict(dict,orient='index').transpose()
    df_epub.columns = df_epub.columns.str.upper()
    return df_epub.transpose()


def get_epub_cover(epub_path):
    ''' Return the cover image file from an epub archive. '''
    
    # Define namespaces
    namespaces = {
                   "calibre":"http://calibre.kovidgoyal.net/2009/metadata",
                   "dc":"http://purl.org/dc/elements/1.1/",
                   "dcterms":"http://purl.org/dc/terms/",
                   "opf":"http://www.idpf.org/2007/opf",
                   "u":"urn:oasis:names:tc:opendocument:xmlns:container",
                   "xsi":"http://www.w3.org/2001/XMLSchema-instance",
                 }

    # We open the epub archive using zipfile.ZipFile():
    with zipfile.ZipFile(epub_path) as z:
        
        # We load "META-INF/container.xml" using lxml.etree.fromString():
        t = etree.fromstring(z.read("META-INF/container.xml"))
        # We use xpath() to find the attribute "full-path":
        '''
        <container version="1.0" xmlns="urn:oasis:names:tc:opendocument:xmlns:container">
          <rootfiles>
            <rootfile full-path="OEBPS/content.opf" ... />
          </rootfiles>
        </container>
        '''
        rootfile_path =  t.xpath("/u:container/u:rootfiles/u:rootfile",
                                             namespaces=namespaces)[0].get("full-path")
        print("Path of root file found: " + rootfile_path)
        
        # We load the "root" file, indicated by the "full_path" attribute of "META-INF/container.xml", using lxml.etree.fromString():
        t = etree.fromstring(z.read(rootfile_path))
        # We use xpath() to find the attribute "content":
        '''
        <metadata xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:opf="http://www.idpf.org/2007/opf">
          ...
          <meta content="my-cover-image" name="cover"/>
          ...
        </metadata>
        '''
        cover_id = t.xpath("//opf:metadata/opf:meta[@name='cover']",
                                    namespaces=namespaces)[0].get("content")
        print("ID of cover image found: " + cover_id)
        
        # We use xpath() to find the attribute "href":
        '''
        <manifest>
            ...
            <item id="my-cover-image" href="images/978.jpg" ... />
            ... 
        </manifest>
        '''
        cover_href = t.xpath("//opf:manifest/opf:item[@id='" + cover_id + "']",
                                         namespaces=namespaces)[0].get("href")
        # In order to get the full path for the cover image, we have to join rootfile_path and cover_href:
        cover_path = os.path.join(os.path.dirname(rootfile_path), cover_href)
        print("Path of cover image found: " + cover_path)
        
        # We return the image
        return z.open(cover_path)

def pdf_metadata_to_df(fname):
    pdf = open(fname, 'rb')
    parser = PDFParser(pdf)
    doc = PDFDocument(parser)
    df_pdf = pd.DataFrame(doc.info)
    df_pdf.columns = df_pdf.columns.str.upper()
    return df_pdf.transpose()


def metadata_to_sql(df):
    1
    return 1

## Excel/CSV file to SQL Database

In [158]:
fname = 'seattle-weather.csv'
schema = 'phpmyadmin'
table_name = ''
query = ''

create_table_sql(query,fname,schema,table_name)

Replacing special characters in table name with "_".
File already exists.
Fullpath: D:\Dataset\Weather_Prediction_Ananth\seattle_weather.csv
Checking in the database.


Table already exists in the database. Do you want to create a new table? [YES/NO] y
Insert new table name. copy


New fullpath: D:/Dataset/Weather_Prediction_Ananth/copy.csv
Importing to database.
Finished.
Execution time: 4.441175 s


C:\Users\Shrek\AppData\Local\Temp\ipykernel_6688\3457088097.py:207: UserWarning: The provided table name 'COPY' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  sql = df.to_sql(name=new_table_name.upper(), con=engine, schema=schema, index=False)


In [146]:
fname = 'seattle-weather.tar.gz'

special_characters = "!@#$%^&*()[]{};:,./<>?\|`~-=_+"

filename = fname.split('.')
print(filename)
# fname_clean = filename[0].translate ({ord(c): "_" for c in special_characters})
fname_clean = filename[0].translate ({ord(c): "_" for c in special_characters})
for i in range(1,len(filename)):
    fname_clean += '.' + filename[i]
print(fname_clean)

dataset_dir = pathlib.Path(r'D:/Dataset')
dataset_dir_list = [item for item in dataset_dir.rglob("*") if item.is_dir()]
dataset_file_list = [item for item in dataset_dir.rglob("*") if item.is_file()]
dataset_dir_array_shape = np.array(dataset_dir_list).shape
dataset_file_array_shape = np.array(dataset_file_list).shape
fullpath = None
is_fullpath = None
for i in range(dataset_dir_array_shape[0]):
    # print(str(dataset_dir_list[i]))
    for j in range(dataset_file_array_shape[0]):
        # print(str(dataset_file_list[j]))
        if os.path.join(str(dataset_dir_list[i]),fname) == str(dataset_file_list[j]):
            folder_path = str(dataset_dir_list[i])
            fullpath = os.path.join(str(dataset_dir_list[i]),fname)
            is_fullpath = os.path.exists(fullpath)
            if table_name == '' or table_name == None:
                name = fname.split('.')
                filename = name[0].split('/')
                table_name = filename[-1]
            if query == '' or table_name == None:
                query = f'''
                            select * from {schema}.{table_name};
                         '''
            fname = folder_path.replace("\\","/")+'/'+fname
            
# Replace special characters in file name with "_"
if any(item in special_characters for item in fname):
    print('File name contains special characters. Renaming file.')
    old_file = os.path.join(folder_path,fname)
    print(old_file)
    new_file = os.path.join(folder_path.replace("\\","/")+'/',fname_clean)
    new_file = folder_path.replace("\\","/")+'/'+fname
    print(new_file)
    # fname = str(old_file.rename(new_file))
    fullpath = fname
    # print(fullpath)

['seattle-weather', 'tar', 'gz']
seattle_weather.tar.gz
File name contains special characters. Renaming file.
D:\Dataset\Weather_Prediction_Ananth\seattle-weather.tar.gz
D:/Dataset/Weather_Prediction_Ananth/seattle-weather.tar.gz


In [127]:
dataset_dir = pathlib.Path(r'D:/Dataset/Books')
dataset_dir_list = [str(item).replace("\\","/") for item in dataset_dir.rglob("*") if item.is_dir()]
dataset_epub_list = [str(item).replace("\\","/") for item in dataset_dir.rglob("*.epub") if item.is_file()]
dataset_pdf_list = [str(item).replace("\\","/") for item in dataset_dir.rglob("*.pdf") if item.is_file()]

name = dataset_epub_list[0].split('.')
filename = name[0].split('/')

# data_dict = {epub_meta}
# print(epub_meta[0])
df_epub = epub_metadata_to_df(dataset_epub_list[0])
display(df_epub)


df_pdf = pdf_metadata_to_df(dataset_pdf_list[0])
display(df_pdf)

,0
IDENTIFIER,a5abd6d1-887f-4afd-b6c3-a3f6539e27c6
TITLE,The Little Prince
AUTHOR_LIST,[Antoine de Saint Exupéry]
SERIES,None
SERIES_INDEX,None
TAG_LIST,[]
LANG,en
DESCRIPTION,
TRANSLATORS,[]
FORMAT,epub


,0
CREATIONDATE,"b""D:20130922181110+08'00'"""
CREATOR,b'DjVuToy'
MODDATE,"b""D:20180429215352+08'00'"""
PRODUCER,b'FreePic2Pdf_Lib - v3.02'


In [100]:
table_name = ''
qwe = 'D:\Dataset\Credit_Card_Approval_Prediction_Seanny\application_record.csv'
sdf = 'seattle-weather.csv'
name = sdf.split('.')
filename = name[0].split('/')
dfg = filename[-1]
# print(name)
asd = dfg.translate ({ord(c): "_" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
print(asd)

# if table_name == '' or table_name == None:
#     # table_name = os.path.splitext(fname)[0]
#     # table_name = pathlib.Path(qwe).stem
#     name = sdf.split('.')
#     filename = name[0].split('/')
#     table_name = filename[-1]
#     print(table_name)
#     print(type(table_name))
#     print([table_name])
#     print(str(table_name))
#     print([str(table_name)])

seattle_weather


In [90]:
my_credentials = get_credentials('my_credentials.txt')
schema = 'phpmyadmin'
table_name = 'application_record'
# query_check = f'''
#                 select count(*) num_cases
#                 from information_schema.tables
#                 where 1=1
#                       and upper(table_schema) = "{schema.upper()}"
#                       and upper(table_name) = "{table_name.upper()}"
#               '''
# print(query_check)
# # Open
# engine = connect_sqlalchemy(my_credentials)

# conn = engine.raw_connection()
# curr = conn.cursor()

# # query_checker = query_check(schema,table_name)
# curr.execute(query_check(schema,table_name))
# # curr.execute(query_check)

curr.close()
engine.dispose()

In [5]:
df = pd.read_csv(r'D:/Dataset/Vehicle_Fuel_Economy_Maharaj/fuel.csv')
df.head(5)

C:\Users\Shrek\AppData\Local\Temp\ipykernel_11244\3140341275.py:10: DtypeWarning: Columns (7,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:/Dataset/Vehicle_Fuel_Economy_Maharaj/fuel.csv')


,vehicle_id,year,make,model,class,drive,transmission,transmission_type,engine_index,engine_descriptor,...,hours_to_charge_ac_240v,composite_city_mpg,composite_highway_mpg,composite_combined_mpg,range_ft1,city_range_ft1,highway_range_ft1,range_ft2,city_range_ft2,highway_range_ft2
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9001,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9005,(FFS) CA model,...,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9002,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9006,(FFS) CA model,...,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0
4,27550,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,NaN,1830,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.0,0.0


In [7]:
df = pd.DataFrame(df)
df

,vehicle_id,year,make,model,class,drive,transmission,transmission_type,engine_index,engine_descriptor,...,hours_to_charge_ac_240v,composite_city_mpg,composite_highway_mpg,composite_combined_mpg,range_ft1,city_range_ft1,highway_range_ft1,range_ft2,city_range_ft2,highway_range_ft2
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9001,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9005,(FFS) CA model,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9002,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9006,(FFS) CA model,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
4,27550,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,NaN,1830,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38108,37564,2017,Volvo,XC60 FWD,Small Sport Utility Vehicle 2WD,Front-Wheel Drive,Automatic (S8),NaN,90,SIDI,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
38109,37547,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),NaN,52,SIDI,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
38110,37548,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),NaN,53,SIDI,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
38111,37703,2017,Volvo,XC90 AWD PHEV,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),NaN,54,SIDI; PHEV,...,0.0,29,32,30,0,0.0,0.0,NaN,13.84,13.3


In [128]:
dataset_dir = pathlib.Path(r'D:/Dataset')
dataset_dir_csv = dataset_dir.rglob("*.csv")
dataset_dir_iter = dataset_dir.rglob("*")
# dataset_dir_list = list(dataset_dir_iter)
dataset_dir_list = [item for item in dataset_dir_iter if item.is_dir()]
dataset_dir_list
fullpath = os.path.join(str(dataset_dir_list[1]),'fuel.csv')
is_fullpath = os.path.exists(os.path.join(str(dataset_dir_list[1]),'fuel.csv'))
if is_fullpath == True:
    df = pd.read_csv(fullpath)
df

C:\Users\Shrek\AppData\Local\Temp\ipykernel_11244\214187510.py:10: DtypeWarning: Columns (7,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fullpath)


,vehicle_id,year,make,model,class,drive,transmission,transmission_type,engine_index,engine_descriptor,...,hours_to_charge_ac_240v,composite_city_mpg,composite_highway_mpg,composite_combined_mpg,range_ft1,city_range_ft1,highway_range_ft1,range_ft2,city_range_ft2,highway_range_ft2
0,26587,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9001,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
1,27705,1984,Alfa Romeo,GT V6 2.5,Minicompact Cars,NaN,Manual 5-Speed,NaN,9005,(FFS) CA model,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
2,26561,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9002,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
3,27681,1984,Alfa Romeo,Spider Veloce 2000,Two Seaters,NaN,Manual 5-Speed,NaN,9006,(FFS) CA model,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
4,27550,1984,AM General,DJ Po Vehicle 2WD,Special Purpose Vehicle 2WD,2-Wheel Drive,Automatic 3-Speed,NaN,1830,(FFS),...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38108,37564,2017,Volvo,XC60 FWD,Small Sport Utility Vehicle 2WD,Front-Wheel Drive,Automatic (S8),NaN,90,SIDI,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
38109,37547,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),NaN,52,SIDI,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
38110,37548,2017,Volvo,XC90 AWD,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),NaN,53,SIDI,...,0.0,0,0,0,0,0.0,0.0,NaN,0.00,0.0
38111,37703,2017,Volvo,XC90 AWD PHEV,Standard Sport Utility Vehicle 4WD,All-Wheel Drive,Automatic (S8),NaN,54,SIDI; PHEV,...,0.0,29,32,30,0,0.0,0.0,NaN,13.84,13.3
